<a href="https://colab.research.google.com/github/ayeonghong928-sudo/AYOUNGHONG/blob/main/%EA%B3%A0%EB%B9%84%EC%96%B4%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("walmart.csv")

In [ ]:
# (1) 데이터 크기 및 컬럼 확인
print("Shape:", df.shape)
print(df.columns)

# (2) 데이터 타입 확인
df.info()

# (3) 결측치 비율 확인 (%)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
print(missing_pct)

# (4) 중복 행 확인
print("Duplicated rows:", df.duplicated().sum())

Shape: (5000, 28)
Index(['transaction_id', 'customer_id', 'product_id', 'product_name',
       'category', 'quantity_sold', 'unit_price', 'transaction_date',
       'store_id', 'store_location', 'inventory_level', 'reorder_point',
       'reorder_quantity', 'supplier_id', 'supplier_lead_time', 'customer_age',
       'customer_gender', 'customer_income', 'customer_loyalty_level',
       'payment_method', 'promotion_applied', 'promotion_type',
       'weather_conditions', 'holiday_indicator', 'weekday',
       'stockout_indicator', 'forecasted_demand', 'actual_demand'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          5000 non-null   int64  
 1   customer_id             5000 non-null   int64  
 2   product_id              5000 non-null   int64  
 3   product_name            50

In [ ]:
# (1) transaction_date를 datetime으로 변환

#transaction_date를 진짜 날짜로 바꿔서 월/요일 등 시간 기반 변수를 만들기 위함
#이후 홀리데이/시즌 분석의 기반

df["transaction_date_parsed"] = pd.to_datetime(
    df["transaction_date"], errors="coerce"
)

# (2) 분석용 시간 파생 변수 생성
df["year_calc"] = df["transaction_date_parsed"].dt.year
df["month_calc"] = df["transaction_date_parsed"].dt.month
df["weekday_calc"] = df["transaction_date_parsed"].dt.day_name()

In [ ]:
# 수요,재고,고객 관련 수치 컬럼을 numeric으로 변환

num_cols = [
    "quantity_sold",
    "unit_price",
    "inventory_level",
    "reorder_point",
    "reorder_quantity",
    "supplier_lead_time",
    "forecasted_demand",
    "actual_demand",
    "customer_age",
    "customer_income"
]

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [ ]:
# (1) 거래 매출 계산
df["transaction_revenue_calc"] = df["unit_price"] * df["quantity_sold"]

# (2) 수요예측 오차 계산
df["forecast_error_calc"] = df["actual_demand"] - df["forecasted_demand"]

# (3) 절대 오차 (#예측이 얼마나 크게 틀렸는지)
df["abs_error_calc"] = df["forecast_error_calc"].abs()

# (4) 비율 오차 # 실제 수요 대비 몇 % 틀렸는지
df["ape_calc"] = df["abs_error_calc"] / df["actual_demand"].replace(0, np.nan)

In [ ]:
# (1) 비교에 필요한 컬럼을 다시 한 번 numeric으로
df["inventory_level"] = pd.to_numeric(df["inventory_level"], errors="coerce")
df["actual_demand"] = pd.to_numeric(df["actual_demand"], errors="coerce")

# (2) 잠재적 품절 위험 계산 (True/False)
df["stockout_risk_calc"] = (
    (df["actual_demand"] > df["inventory_level"]) &
    df["actual_demand"].notna() &
    df["inventory_level"].notna()
)

# (3) Boolean → 0/1 변환
df["stockout_risk_calc"] = df["stockout_risk_calc"].astype(int)

In [ ]:
# 프로모션 변수 정리

#(1) promotion_applied를 분석용 0/1로 정리
df["promotion_applied_clean"] = df["promotion_applied"].map(
    {True: 1, False: 0}
).astype("Int64")

# (2) promotion_type은 원본 유지 + 분석
df["promotion_type_clean"] = df["promotion_type"].astype("string")

# 프로모션 미적용이면 No Promotion
df.loc[df["promotion_applied"] == False, "promotion_type_clean"] = "No Promotion"

# 적용됐는데 타입이 없으면 Unspecified
df.loc[
    (df["promotion_applied"] == True) &
    (df["promotion_type_clean"].isna()),
    "promotion_type_clean"
] = "Unspecified"

In [ ]:
#가격대 구간화

df["price_segment_calc"] = pd.qcut(
    df["unit_price"],
    q=4,
    labels=["Low", "Mid-Low", "Mid-High", "High"]
)

In [ ]:
#최종 점검
print("Final shape:", df.shape)

# 주요 파생 변수 결측 확인
check_cols = [
    "transaction_revenue_calc",
    "forecast_error_calc",
    "abs_error_calc",
    "stockout_risk_calc"
]
print(df[check_cols].isna().mean())

Final shape: (5000, 40)
transaction_revenue_calc    0.0
forecast_error_calc         0.0
abs_error_calc              0.0
stockout_risk_calc          0.0
dtype: float64


In [ ]:
#CSV 저장
from google.colab import files
files.download("walmart_preprocessed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>